In [26]:
import cv2
import numpy as np
import sys
import os.path
from PIL import Image

In [27]:
orig_img = cv2.imread('download.png')

In [28]:
img = cv2.copyMakeBorder(orig_img, 50, 50 ,50 ,50 ,cv2.BORDER_CONSTANT)

In [29]:
img_y = len(img)
img_x = len(img[0])

In [30]:
print("Dimensions of image: "+ str(len(img)) +" x " + str(len(img[0])))

Dimensions of image: 279 x 382


In [31]:
# blue, green, red = cv2.split(img)

In [32]:
# # Run canny edge detection on each channel
# blue_edges = cv2.Canny(blue, 200, 250)
# green_edges = cv2.Canny(green, 200, 250)
# red_edges = cv2.Canny(red, 200, 250)

In [33]:
# Join edges back into image
# edges = blue_edges | green_edges | red_edges

In [34]:
edges = cv2.Canny(img, 200, 250)

In [35]:
_, contours, hierarchy = cv2.findContours(edges.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

In [36]:
hierarchy[0].shape

(265, 4)

In [37]:
processed = edges.copy()
rejected = edges.copy()

In [38]:
def ii(xx, yy):
    global img, img_y, img_x
    if yy >= img_y or xx >= img_x:
        #print "pixel out of bounds ("+str(y)+","+str(x)+")"
        return 0
    pixel = img[yy][xx]
    return 0.30 * pixel[2] + 0.59 * pixel[1] + 0.11 * pixel[0]

In [39]:
def keep_box(contour):
    xx, yy, w_, h_ = cv2.boundingRect(contour)

    # width and height need to be floats
    w_ *= 1.0
    h_ *= 1.0

    # Test it's shape - if it's too oblong or tall it's probably not a real character
    if w_ / h_ < 0.1 or w_ / h_ > 10:
        return False
    
    # check size of the box
    if ((w_ * h_) > ((img_x * img_y) / 5)) or ((w_ * h_) < 15):
        return False

    return True


In [40]:
def connected(contour):
    first = contour[0][0]
    last = contour[len(contour) - 1][0]
    return abs(first[0] - last[0]) <= 1 and abs(first[1] - last[1]) <= 1


In [41]:
def keep(contour):
    return keep_box(contour) and connected(contour)

In [55]:
def get_parent(index, h_):
    parent = h_[index][1]
    while not keep(c(parent)) and parent > 0:
        parent = h_[parent][1]

    return parent


In [51]:
def is_child(index, h_):
    return get_parent(index, h_) > 0


In [52]:
def c(index):
    global contours
    return contours[index]

In [45]:
def include_box(index, h_, contour):
    print (str(index) + ":")
    if is_child(index, h_):
            print ("\tIs a child")
            print ("\tparent " + str(get_parent(index, h_)) + " has " + str(
                count_children(get_parent(index, h_), h_, contour)) + " children")
            print( "\thas " + str(count_children(index, h_, contour)) + " children")

    
  
        
    if is_child(index, h_) and count_children(get_parent(index, h_), h_, contour) <= 2:
        return False

    if count_children(index, h_, contour) > 2:
        return False

 
    return True


In [46]:
def count_children(index, h_, contour):
    # No children
    if h_[index][2] < 0:
        return 0
    else:
        #If the first child is a contour we care about
        # then count it, otherwise don't
        if keep(c(h_[index][2])):
            count = 1
        else:
            count = 0

            # Also count all of the child's siblings and their children
        count += count_siblings(h_[index][2], h_, contour, True)
        return count


# Quick check to test if the contour is a child
def is_child(index, h_):
    return get_parent(index, h_) > 0


# Get the first parent of the contour that we care about
def get_parent(index, h_):
    parent = h_[index][3]
    while not keep(c(parent)) and parent > 0:
        parent = h_[parent][3]

    return parent





In [47]:
# Count the number of relevant siblings of a contour
def count_siblings(index, h_, contour, inc_children=False):
    # Include the children if necessary
    if inc_children:
        count = count_children(index, h_, contour)
    else:
        count = 0

    # Look ahead
    p_ = h_[index][0]
    while p_ > 0:
        if keep(c(p_)):
            count += 1
        if inc_children:
            count += count_children(p_, h_, contour)
        p_ = h_[p_][0]

    # Look behind
    n = h_[index][1]
    while n > 0:
        if keep(c(n)):
            count += 1
        if inc_children:
            count += count_children(n, h_, contour)
        n = h_[n][1]
    return count

In [48]:
# These are the boxes that we are determining
keepers = []

In [56]:
for index_, contour_ in enumerate(contours):
   
    x, y, w, h = cv2.boundingRect(contour_)

    # Check the contour and it's bounding box
    if keep(contour_) and include_box(index_, hierarchy, contour_):
        keepers.append([contour_, [x, y, w, h]])
        
        cv2.rectangle(processed, (x, y), (x + w, y + h), (100, 100, 100), 1)
        cv2.putText(processed, str(index_), (x, y - 5), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255))
    
    else:
        cv2.rectangle(rejected, (x, y), (x + w, y + h), (100, 100, 100), 1)
        cv2.putText(rejected, str(index_), (x, y - 5), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255))


4:


IndexError: index 4 is out of bounds for axis 0 with size 1

In [ ]:
new_image = edges.copy()
new_image.fill(255)
boxes = []


In [ ]:
for index_, (contour_, box) in enumerate(keepers):

    # Find the average intensity of the edge pixels to
    # determine the foreground intensity
    fg_int = 0.0
    for p in contour_:
        fg_int += ii(p[0][0], p[0][1])

    fg_int /= len(contour_)
    
    print ("FG Intensity for #%d = %d" % (index_, fg_int))

    
    x_, y_, width, height = box
    bg_int = [# bottom left corner 3 pixels
            ii(x_ - 1, y_ - 1),
            ii(x_ - 1, y_),
            ii(x_, y_ - 1),

            # bottom right corner 3 pixels
            ii(x_ + width + 1, y_ - 1),
            ii(x_ + width, y_ - 1),
            ii(x_ + width + 1, y_),

            # top left corner 3 pixels
            ii(x_ - 1, y_ + height + 1),
            ii(x_ - 1, y_ + height),
            ii(x_, y_ + height + 1),

            # top right corner 3 pixels
            ii(x_ + width + 1, y_ + height + 1),
            ii(x_ + width, y_ + height + 1),
            ii(x_ + width + 1, y_ + height)]        

    # Find the median of the background
    # pixels determined above
    bg_int = np.median(bg_int)

    print ("BG Intensity for #%d = %s" % (index_, repr(bg_int)))

    # Determine if the box should be inverted
    if fg_int >= bg_int:
        fg = 255
        bg = 0
    else:
        fg = 0
        bg = 255

        # Loop through every pixel in the box and color the
        # pixel accordingly
    for x in range(x_, x_ + width):
        for y in range(y_, y_ + height):
            
            if y >= img_y or x >= img_x:                
                continue
                
            if ii(x, y) > fg_int:
                new_image[y][x] = bg
                
            else:
                new_image[y][x] = fg

# blur a bit to improve ocr accuracy
new_image = cv2.blur(new_image, (2, 2))
cv2.imwrite(output_file, new_image)

cv2.imwrite('edges.png', edges)
cv2.imwrite('processed.png', processed)
cv2.imwrite('rejected.png', rejected)

In [ ]:
import matplotlib.pyplot as plt
random = new_image
plt.imshow(random)
plt.show()

In [ ]:
im = Image.fromarray(new_image)
im.save('test.png')

In [ ]:
import pytesseract

pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files (x86)/Tesseract-OCR/tesseract'

In [30]:
result = pytesseract.image_to_string(Image.open("test.png"))
print(result)

4 WkiJre €99 Bread

A good, basic white bread.

with

I. 21/2 cups lukewarm water
2 packet; es dry yeast
1/4 cup honey
1 cup dry mil»
2 eggs, beaten
4 cups unbleached white ﬂour

II. 4 teaspoons salt
1/3 cup butter or margarine
3 cups or more unbleached white ﬂour for forming the dough
1 cup (approx.) white ﬂour for kneading

Proceed with the directions for recipe #1, adding the beaten eggs afte
stirring in the dry milk.

ad

is moister and chewier than the white e ;g bread.

rid,
. 1" er
ast
J
V -
